
----
# Cuaderno 11 - Automatización del flujo de trabajo con Pipelines y GridSearchCV
## Ariel Palazzesi - 2026
----

Hasta ahora hemos entrenado modelos y procesado datos siguiendo pasos separados: limpieza, escalado, entrenamiento, evaluación, etc. En esta oportunidad aprenderemos a **encadenar esos pasos de forma automática y ordenada** utilizando `Pipeline`, una herramienta de `scikit-learn` que nos permite construir flujos de trabajo reproducibles y seguros.

Además, introduciremos `GridSearchCV`, una técnica para buscar **la mejor combinación de hiperparámetros** de forma sistemática y validada, integrándola dentro del pipeline.

Este cuaderno incluye:
- Construcción de un pipeline básico (transformación + modelo).
- Evaluación del pipeline con validación cruzada.
- Optimización de parámetros con GridSearchCV.

¿Comenzamos?

## Pipelines y búsqueda de hiperparámetros en scikit-learn

Hasta acá fuimos haciendo el flujo de trabajo “a mano”: primero limpiamos, después escalamos, luego entrenamos, y finalmente evaluamos. El problema de ese enfoque es que es fácil **desordenarse**, repetir pasos o, peor, cometer errores sutiles (por ejemplo, calcular un escalado usando datos de test). Para evitar eso, scikit-learn ofrece dos herramientas que suelen ir juntas: **Pipeline** y **GridSearchCV**.

Veamos en que consisten y como usarlas.

---

### 1) Construcción de un pipeline básico (transformación + modelo)

Un **Pipeline** es una cadena de pasos donde cada paso hace una parte del trabajo en el orden correcto. La idea es simple: *primero transformo los datos, después entreno el modelo*, y todo queda empaquetado como si fuera un único “modelo”.

Un pipeline típico tiene esta forma conceptual:

1. **Transformación** (ej.: escalado, imputación de nulos, selección de variables, etc.)
2. **Modelo** (ej.: regresión logística, KNN, Random Forest, etc.)

Lo importante es que el pipeline asegura que:

* en `fit()` las transformaciones se “aprenden” **solo con el set de entrenamiento** (por ejemplo, media y desvío para escalar),
* y en `predict()` se aplican esas mismas transformaciones al nuevo conjunto de datos de forma consistente.

Esto hace el flujo más **reproducible** (siempre se ejecuta igual) y más **seguro** (reduce fugas de información entre train y test).

---

### 2) Evaluación del pipeline con validación cruzada

Una vez que tenemos listo el pipeline, lo podemos evaluar con validación cruzada igual que cualquier modelo, pero con una ventaja enorme: **cada fold repite correctamente todo el proceso**.

En validación cruzada, el flujo correcto es:

* para cada pliegue:

  * ajustar transformaciones con el entrenamiento del pliegue,
  * entrenar el modelo con ese entrenamiento,
  * aplicar transformaciones al set de validación del pliegue,
  * evaluar en validación.

Si hacemos transformaciones “afuera” del pipeline, existe el riesgo de que queden calculadas usando todo el dataset, y eso sesga la evaluación. Con Pipeline, scikit-learn lo maneja de forma automática y correcta.

---

### 3) Optimización de parámetros con GridSearchCV

En Machine Learning, muchos modelos y transformaciones tienen **hiperparámetros**, que son configuraciones que elegimos antes de entrenar (por ejemplo, el valor de `k` en KNN, la profundidad máxima en un árbol, el `C` en regresión logística, etc.). La pregunta típica es: *¿qué valores convienen?*

**GridSearchCV** es una herramienta que prueba sistemáticamente combinaciones de hiperparámetros y elige la mejor según una métrica (accuracy, F1, RMSE, etc.), usando validación cruzada para que la elección sea **robusta**.

La idea conceptual es:

1. Definimos un conjunto de parámetros posibles (una “grilla”).
2. Para cada combinación:

   * entrena y evalúa con validación cruzada,
   * calcula el rendimiento promedio.
  
3. Elige la combinación con mejor rendimiento promedio.
4. Devuelve un “mejor estimador” listo para usar.

Lo más potente es que GridSearchCV se integra perfecto con Pipeline: es posible buscar hiperparámetros **tanto del modelo como de las transformaciones**, y todo se evalúa sin fugas, fold por fold.

---

### ¿Por qué Pipeline + GridSearchCV es un combo tan usado?

Porque proporciona un flujo:

* **ordenado** (pasos explícitos),
* **repetible** (mismo proceso siempre),
* **correcto** (sin data leakage),
* **optimizado** (encuentra buenos hiperparámetros con validación cruzada),
* y fácil de compartir o llevar a producción.




## Carga del dataset

Vamos a trabajar nuevamente con el dataset del Titanic. Este dataset ya nos es familiar: contiene información sobre los pasajeros, como edad, tarifa, clase, género, entre otros.

Nuestro objetivo será predecir la variable `Survived`, indicando si la persona sobrevivió al hundimiento, utilizando un flujo de trabajo encapsulado en un pipeline.

Primero cargamos los datos y exploramos brevemente su estructura.


In [ ]:
# Importamos librerías necesarias
import pandas as pd

# Cargamos el archivo desde el entorno de Colab
df = pd.read_csv("Titanic-Dataset.csv")

# Mostramos las primeras filas
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preparación de los datos

Vamos a seleccionar un conjunto de variables que consideramos útiles para predecir la supervivencia (`Survived`). En este caso trabajaremos únicamente con columnas numéricas para facilitar el uso del pipeline.

Variables seleccionadas:
- `Pclass`: Clase del pasajero (1°, 2°, 3°).
- `Age`: Edad.
- `SibSp`: Cantidad de hermanos/esposas a bordo.
- `Parch`: Cantidad de padres/hijos a bordo.
- `Fare`: Tarifa pagada.

La variable objetivo será `Survived`.

Antes de entrenar, eliminaremos las filas con valores nulos en las columnas seleccionadas.


In [ ]:
# Seleccionamos variables numéricas relevantes
columnas = ["Pclass", "Age", "SibSp", "Parch", "Fare", "Survived"]
df_filtrado = df[columnas].dropna()

# Separamos X (features) e y (variable objetivo)
X = df_filtrado.drop("Survived", axis=1)
y = df_filtrado["Survived"]

# Verificamos dimensiones
X.shape, y.shape


((714, 5), (714,))

## División del dataset en entrenamiento y prueba

Antes de construir el pipeline, vamos a separar los datos en dos conjuntos:

- **Entrenamiento (train)**: será usado para ajustar el modelo y aplicar validación cruzada.
- **Prueba (test)**: será utilizado exclusivamente para evaluar el rendimiento final del modelo ya entrenado.

Esto nos permite obtener una estimación más realista sobre cómo se comportaría el modelo con datos nuevos.


In [ ]:
from sklearn.model_selection import train_test_split

# Dividimos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificamos tamaños
X_train.shape, X_test.shape


((571, 5), (143, 5))

En este caso, el 80% del dataset (571 registros) se destina al entrenamiento y el 20% restante (143 registros) a la prueba.

Esta proporción es común en muchos proyectos y nos asegura que el modelo tiene suficiente información para aprender, sin dejar de reservar un conjunto representativo para validar su comportamiento con datos no vistos.


## Construcción del pipeline (escalado + modelo)



1. **Escalado de variables** usando `StandardScaler`, para que todas las variables numéricas estén en la misma escala.

2. **Entrenamiento de un modelo supervisado**, en este caso una regresión logística.

Encapsular este flujo dentro de un pipeline nos asegura que todas las transformaciones se realicen correctamente, y nos permitirá evaluar y reutilizar el modelo con seguridad.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Definimos el pipeline
pipeline = Pipeline([
    ('escalado', StandardScaler()),
    ('modelo', LogisticRegression())
])

# Entrenamos el pipeline completo
pipeline.fit(X_train, y_train)


Pipeline(steps=[('escalado', StandardScaler()),
                ('modelo', LogisticRegression())])

## ¿Qué estamos viendo en esta representación?

La imagen que aparece como salida de la celda muestra una **visualización estructurada del objeto `Pipeline`**, generada automáticamente por Google Colab.

Cada recuadro representa un paso del pipeline:

- El primer bloque, `StandardScaler()`, indica que los datos serán **escalados** antes de entrenar el modelo. Esto es importante porque muchos algoritmos, incluida la regresión logística, son sensibles a la escala de las variables.
  
- El segundo bloque, `LogisticRegression()`, es el **modelo de clasificación** que será entrenado sobre los datos ya transformados por el paso anterior.

Este diagrama refleja la estructura jerárquica del pipeline: los datos ingresan por la parte superior y fluyen hacia abajo, pasando por cada paso en orden. Es una excelente forma de confirmar visualmente que el flujo fue definido correctamente.


## Evaluación del rendimiento del pipeline

Una vez entrenado, el pipeline se comporta como cualquier modelo de scikit-learn: podemos usar `.predict()` para hacer predicciones y luego comparar los resultados con los valores reales del conjunto de prueba.

En este caso, evaluaremos el modelo utilizando el reporte de clasificación (`classification_report`), que nos ofrece métricas clave como precisión, recall, F1-score y exactitud.


In [ ]:
from sklearn.metrics import classification_report

# Realizamos predicciones con el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Mostramos el reporte de clasificación
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.71      0.85      0.77        87
           1       0.67      0.46      0.55        56

    accuracy                           0.70       143
   macro avg       0.69      0.66      0.66       143
weighted avg       0.69      0.70      0.69       143



## Interpretación del rendimiento

El modelo entrenado mediante el pipeline alcanzó una **exactitud del 70 %** sobre el conjunto de prueba. A primera vista, es un resultado aceptable para un primer intento con pocos pasos de ajuste.

Sin embargo, al analizar más en detalle:

- El modelo predice bastante bien la clase **0** (personas que no sobrevivieron), con un recall de 0.85.
- Pero tiene más dificultades con la clase **1** (personas que sí sobrevivieron), alcanzando un recall de solo 0.46.  
  Esto significa que **casi la mitad de los casos positivos reales no fueron detectados**.

Este desequilibrio puede estar relacionado con:
- La distribución de clases (desbalanceo),
- La sencillez del modelo y su configuración por defecto,
- O la necesidad de ajustar mejor los hiperparámetros.

En la siguiente sección, vamos a aplicar **validación cruzada** y luego utilizaremos **GridSearchCV** para buscar configuraciones que mejoren el rendimiento global del pipeline.


## Validación cruzada con `cross_val_score`

En lugar de entrenar y evaluar el modelo una sola vez, vamos a aplicar **validación cruzada** para estimar su rendimiento de forma más estable. Utilizaremos `cross_val_score`, que divide automáticamente el conjunto de entrenamiento en varias particiones (folds), entrena el pipeline en algunas y evalúa en otras.

Esto nos permitirá ver **cómo se comporta el pipeline en distintos subconjuntos del dataset**, y anticipar posibles problemas de sobreajuste o variabilidad en el rendimiento.

Usaremos 5 particiones (5-fold cross-validation) y evaluaremos el modelo con la métrica de exactitud (`accuracy`).


In [ ]:
from sklearn.model_selection import cross_val_score

# Evaluamos el pipeline con validación cruzada de 5 folds
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

# Mostramos los resultados
print("Puntajes en cada fold:", scores)
print(f"Promedio de accuracy: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")


Puntajes en cada fold: [0.75652174 0.66666667 0.69298246 0.6754386  0.72807018]
Promedio de accuracy: 0.7039
Desviación estándar: 0.0337


## Resultados de la validación cruzada

El pipeline fue evaluado utilizando validación cruzada con 5 particiones (5-fold).  
El promedio de exactitud obtenido fue de **70.39%**, con una desviación estándar de **±3.37%**.

> Esto sugiere que el modelo tiene un rendimiento relativamente estable en diferentes subconjuntos de datos, lo que indica **buena generalización**.

Sin embargo, los valores varían entre 66% y 75%, lo que también deja espacio para mejorar el modelo ajustando algunos de sus hiperparámetros.

En la siguiente sección vamos a introducir `GridSearchCV`, que nos permitirá **buscar automáticamente la mejor configuración posible** del modelo dentro del pipeline.


## Optimización con `GridSearchCV`

Hasta ahora, nuestro modelo usó la configuración por defecto de `LogisticRegression`. Pero este modelo tiene varios hiperparámetros que pueden influir en su rendimiento. Por ejemplo:

- `C`: controla el nivel de regularización (inversamente proporcional).  
  Valores bajos implican mayor regularización; valores altos, menos.

Para no elegir valores “a ojo”, podemos usar `GridSearchCV`, una herramienta que:

1. Prueba múltiples combinaciones de hiperparámetros,
2. Evalúa cada combinación con validación cruzada,
3. Selecciona automáticamente la mejor opción.

La gran ventaja es que `GridSearchCV` se puede aplicar **sobre un pipeline completo**, respetando todo el flujo de procesamiento.


In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos el rango de valores a probar para el hiperparámetro C
param_grid = {
    'modelo__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

# Creamos el GridSearchCV sobre el pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Ejecutamos la búsqueda
grid_search.fit(X_train, y_train)

# Mostramos la mejor configuración encontrada
print("Mejor valor de C:", grid_search.best_params_['modelo__C'])
print(f"Mejor accuracy promedio (validación cruzada): {grid_search.best_score_:.4f}")


Mejor valor de C: 0.01
Mejor accuracy promedio (validación cruzada): 0.7058


## Resultados de la búsqueda con `GridSearchCV`

Aplicamos `GridSearchCV` sobre nuestro pipeline para optimizar el hiperparámetro `C` del modelo de regresión logística.

* **Mejor valor encontrado:** `C = 0.01`  
* **Mejor accuracy promedio (cross-validation):** 70.58%

Esto indica que, al aplicar un mayor nivel de regularización, el modelo logra un rendimiento ligeramente superior y más generalizable.  
Aunque la mejora respecto al valor por defecto es modesta, demuestra el valor de automatizar la búsqueda de configuraciones, especialmente en modelos más complejos.

En este caso, el pipeline con escalado + modelo optimizado representa una **solución completa, estructurada y reproducible**.


## Conclusión y cierre

En este cuaderno dimos un paso importante en la organización y profesionalización de nuestros modelos de machine learning.

* Construimos un **pipeline completo** que encadena:
  - el preprocesamiento de datos (escalado),
  - el modelo supervisado (regresión logística),
  - y todas las operaciones necesarias para hacer predicciones.

* Evaluamos el pipeline mediante **validación cruzada**, lo que nos permitió estimar su rendimiento de forma más confiable, sin depender de una única partición del dataset.

* Luego utilizamos **GridSearchCV** para explorar distintas configuraciones del modelo y encontrar automáticamente la que ofrecía mejores resultados. De esta manera, logramos un flujo de trabajo más sólido, preciso y repetible.

> Automatizar el flujo con `Pipeline` y optimizar con `GridSearchCV` no solo mejora la calidad de los modelos, sino que también **reduce errores, facilita la colaboración y prepara nuestros proyectos para el mundo real**.

Este enfoque nos permitirá enfrentar problemas más complejos, escalar nuestras soluciones y mantener un estándar profesional en el desarrollo de modelos de aprendizaje automático.

